In [11]:
from typing import Type

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper


llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-0125")

class IssueResearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Research about the world war"
    )


class IssueDDGSResearchTool(BaseTool):
    name = "IssueDDGSResearchTool"
    description = """
    Use this tool to find the Issue using DuckDuckGoSearch.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        IssueResearchToolArgsSchema
    ] = IssueResearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        results = ddg.run(query)

        with open('ddgs.txt', 'w', encoding='utf-8') as file:
            file.write(results)
        return results

class IssueWikipediaResearchTool(BaseTool):
    name = "IssueWikipediaResearchTool"
    description = """
    Use this tool to find the Issue using Wikipedia.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        IssueResearchToolArgsSchema
    ] = IssueResearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        results = wikipedia.run(query)

        with open('wikipedia.txt', 'w', encoding='utf-8') as file:
            file.write(results)
        return results
    

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        IssueDDGSResearchTool(),
        IssueWikipediaResearchTool(),
    ],
)


prompt = "Research about the XZ backdoor"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I should start by using the IssueDDGSResearchTool to search for information on the XZ backdoor.
Action: IssueDDGSResearchTool
Action Input: XZ backdoor
Observation: [snippet: The XZ Backdoor: Everything You Need to Know. Details are starting to emerge about a stunning supply chain attack that sent the open source software community reeling. On Friday, a lone Microsoft ..., title: The XZ Backdoor: Everything You Need to Know | WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/], [snippet: The version of XZ should be carefully checked, as versions 5.6.0 and 5.6.1 contain the backdoor. It is advised to revert to a previous known safe version of XZ Utils, such as 5.4. Software supply ..., title: XZ Utils Supply Chain Attack: A Threat Actor Spent Two Years to ..., link: https://www.techrepublic.com/article/xz-backdoor-linux/], [snippet: That operation matches the style of the XZ Utils backdoor far more than the cruder su

{'input': 'Research about the XZ backdoor',
 'output': 'The XZ backdoor is a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024. It targets Debian- and RPM-based systems running on the x86-64 architecture, giving an attacker remote code execution capabilities.'}